In [186]:
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
from selenium import webdriver
import pandas as pd
import string
import time
import os


In [187]:
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")

chromedriver_autoinstaller.install()

'C:\\Users\\H P\\Envs\\scrapperenv\\lib\\site-packages\\chromedriver_autoinstaller\\104\\chromedriver.exe'

In [188]:
def process_div0(div):
    name = number = license = "" 
    try:
        h3           = div.find_element_by_tag_name("h3")
        b            = h3.find_element_by_tag_name("b")
        name, number = b.text.split("#")
    except:pass
    try:license = div.find_element_by_xpath("//div//p[@class='nostyle']//span").text
    except:pass
    
    return name, number, license

def process_div1(div):
    data = {"Address":"Not Available", "Phone":"Not Available", "Fax":"Not Available", "Email":"Not Available", "Website":"Not Available"}
    for p in div.find_elements_by_tag_name("p"):
        try:
            attrs = p.text.split("|")
            for attr in attrs:
                try:
                    key, val = attr.split(":")
                    data[key.strip()] = val.strip()
                except:
                    pass
        except:
            pass
    return data

def save_data(data, ch):
    folder = "./files"
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    file = os.path.join(folder, f"{ch}.xlsx")
    df = pd.DataFrame(data)
    df.to_excel(file,  index=False)

In [189]:
FIRST_PAGE_LOAD_SLEEP = 3
TR_LOAD_SLEEP         = 2

TABLE_RETRY       = 3
TABLE_RETRY_SLEEP = 1

driver  = webdriver.Chrome(options=opt)
driver1 = webdriver.Chrome(options=opt)


## To Scrap Single Chars 

In [190]:
chars   = string.ascii_lowercase

In [ ]:
for char in chars:
    data_items = []
    url        = f"https://apps.calbar.ca.gov/attorney/LicenseeSearch/QuickSearch?FreeText={char}&SoundsLike=false"
    driver.get (url)
    time.sleep (FIRST_PAGE_LOAD_SLEEP)
    
    while True:
        count = 0
        hrefs = None
        trows = None
        while count < TABLE_RETRY:
            try:
                table      = driver.find_element_by_id("tblAttorney")
                table_body = table.find_element_by_tag_name("tbody")
                trows      = table_body.find_elements_by_tag_name("tr")
                #hrefs      = [tr.find_element_by_tag_name("a").get_attribute("href") for tr in trows]
                break
            except:
                count += 1
                time.sleep(TABLE_RETRY_SLEEP)

        if trows:
            for tr in trows:
                href = tr.find_element_by_tag_name("a").get_attribute("href")
                name = tr.find_element_by_tag_name("a").text

                tds = tr.find_elements_by_tag_name("td")
                try:license = tds[1].find_element_by_tag_name("span").text
                except:license = tds[1].text
                number    = tds[2].text
                if license == "Active":
                    driver1.get(href)
                    time.sleep(TR_LOAD_SLEEP)

                    try:
                        item = {"Address":"Not Available", "Phone":"Not Available", "Fax":"Not Available", "Email":"Not Available", "Website":"Not Available"}
                        divs = driver1.find_elements_by_xpath("//div[@style='margin-top:1em;']")
                        if len(divs) == 2:
                            #name, number, license = process_div0(divs[0])
                            item                  = process_div1(divs[1])

                    except:pass

                    item['Attorney Name'] = name
                    item['Number']        = number
                    item['License Status']= license
                    data_items.append     (item)
                
        try:
            next_btn = driver.find_element_by_id("tblAttorney_next")
            if "disabled" in next_btn.get_attribute("class"):
                break
            else:
                next_btn.click() 
        except:
            break
    
    save_data(data_items, char)
    
        
    


## To Scrap Double Chars

In [ ]:
chars   = string.ascii_lowercase

In [ ]:
for outer_char in chars:
    for inner_char in chars:
        char       = outer_char +  inner_char
        data_items = []
        url        = f"https://apps.calbar.ca.gov/attorney/LicenseeSearch/QuickSearch?FreeText={char}&SoundsLike=false"
        driver.get (url)
        time.sleep (FIRST_PAGE_LOAD_SLEEP)

        while True:
            count = 0
            hrefs = None
            trows = None
            while count < TABLE_RETRY:
                try:
                    table      = driver.find_element_by_id("tblAttorney")
                    table_body = table.find_element_by_tag_name("tbody")
                    trows      = table_body.find_elements_by_tag_name("tr")
                    #hrefs      = [tr.find_element_by_tag_name("a").get_attribute("href") for tr in trows]
                    break
                except:
                    count += 1
                    time.sleep(TABLE_RETRY_SLEEP)

            if trows:
                for tr in trows:
                    href = tr.find_element_by_tag_name("a").get_attribute("href")
                    name = tr.find_element_by_tag_name("a").text

                    tds = tr.find_elements_by_tag_name("td")
                    try:license = tds[1].find_element_by_tag_name("span").text
                    except:license = tds[1].text
                    number    = tds[2].text
                    
                    if license == "Active":
                        driver1.get(href)
                        time.sleep(TR_LOAD_SLEEP)

                        try:
                            item = {"Address":"Not Available", "Phone":"Not Available", "Fax":"Not Available", "Email":"Not Available", "Website":"Not Available"}
                            divs = driver1.find_elements_by_xpath("//div[@style='margin-top:1em;']")
                            if len(divs) == 2:
                                #name, number, license = process_div0(divs[0])
                                item                  = process_div1(divs[1])

                        except:pass

                        item['Attorney Name'] = name
                        item['Number']        = number
                        item['License Status']= license
                        data_items.append     (item)

            try:
                next_btn = driver.find_element_by_id("tblAttorney_next")
                if "disabled" in next_btn.get_attribute("class"):
                    break
                else:
                    next_btn.click() 
            except:
                break

        save_data(data_items, char)
    
        
    
